In [1]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from googletrans import Translator
from langdetect import detect




In [2]:
df = pd.read_csv("Dataframe_Wrangled_NoDuplicates.csv")
df.head()

,id,conversations
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom..."
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow..."
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u..."
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult..."


In [3]:
print(len(df))

23277


In [4]:
# parse conversations
text_list = []
text_arrays = []
for conversation in df["conversations"]:
    text = ""
    text_arr = []
    parsed_conversation = eval(conversation)
    for dialogue in parsed_conversation:
        text += dialogue["value"] + " "
        text_arr.append(dialogue["value"])
    text_list.append(text)
    text_arrays.append(text_arr)

# create column with the conversation text
df["text"] = text_list
df["values"] = text_arrays
        


In [5]:
df.head()


,id,conversations,text,values
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ..."
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...


In [7]:
languages = []
for value in df["values"]:
    try:
        lang = detect(value[0])
        languages.append(lang)
    except:
        try:
            lang = detect(value[1])
            languages.append(lang)
        except:
            print("error2")
            print(value[1])


df["lang"] = languages


In [8]:
df.head()

,id,conversations,text,values,lang
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...,en
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ...",en
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...,en
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...,es
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...,en


In [15]:
df_shortened = df[['id', 'values', 'lang']]
df_shortened.to_csv('file_name.csv', index=False, encoding='utf-8')


In [10]:
df["lang"].value_counts()

lang
en       18368
ko         835
zh-cn      618
fr         523
es         457
ja         239
ca         188
it         183
pt         162
vi         160
de         142
no         138
nl         128
ru         126
ro         111
sw          88
et          77
zh-tw       72
id          62
af          60
cy          59
pl          57
fi          56
da          55
tl          36
sv          36
so          33
tr          32
hr          32
sl          21
uk          20
sk          19
th          16
cs          13
lt          12
he          10
hu           8
mk           6
bg           6
sq           4
fa           3
lv           2
ar           2
el           2
Name: count, dtype: int64

In [26]:
translator = Translator()

async def translate(value):
      result = await translator.translate(value, dest="en")
      return result

translated_values = []
for index, item in df["values"].items():
    if df["lang"][index] != "en":
        translated_conversation = []
        for element in item:
            a=re.compile("<.*?>")
            element = a.sub('', element)
            if len(element) > 15000:
                chunks = []
                for i in range(0, len(element), 15000):
                    chunks.append(element[i:i + 15000])
                
                translated_chunks = []
                for chunk in chunks:
                    result = await translate(chunk)
                    translated_chunks.append(result.text)
                
                result = " ".join(translated_chunks)
                translated_conversation.append(result)

            else:
                result = await translate(element)
                translated_conversation.append(result.text)
                
        translated_values.append(translated_conversation)
    else:
        translated_values.append(item)

df["translated_values"] = translated_values


# 67 conversations exceed 15,000


CancelledError: 

In [22]:
print(df["values"][17010])
df.loc[df["id"] == "XQHmkl5"]["conversations"]


['hello', '<div><p>Hello! How can I help you today?</p></div>', 'I need you read the following pdf', "<div><p>I'm sorry, but I am unable to read or access any PDF documents or files. However, I can try to help you with any questions or problems you might have related to the content of the PDF. Please let me know how I can assist you.</p></div>", 'ok can you read the following ', '<div><p>Certainly! Please go ahead and provide the text that you would like me to read.</p></div>', 'PVCS Version Manager\nPCLI User’s Guide and Reference\nCopyright © 2021 Micro Focus or one of its affiliates.\nThe only warranties for products and services of Micro Focus and its affiliates and licensors\n(“Micro Focus”) are as may be set forth in the express warranty statements accompanying such\nproducts and services. Nothing herein should be construed as constituting an additional\nwarranty. Micro Focus shall not be liable for technical or editorial errors or omissions contained\nherein. The information con

12    [{'from': 'human', 'value': '相関係数は1つの変数ともう1つの変...
Name: conversations, dtype: object

In [6]:
corpus = df["text"]

In [8]:
def preprocess_text(text):
    text = str(text)
    # remove angle brackets and all characters inside angle bracket
    a=re.compile("<.*?>")
    text = a.sub('', text)
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # convert to lowercase
    text = text.lower()
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords
    # tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens

# preprocess all text
preprocessed_text = [' '.join(preprocess_text(document)) for document in corpus]

In [12]:
tokens = word_tokenize("相関係数は1つの変数ともう1つの変")
print(tokens)

tokens = word_tokenize("hello there")
print(tokens)

['相関係数は1つの変数ともう1つの変']
['hello', 'there']


In [9]:
df["preprocessed_text"] = preprocessed_text

In [10]:
df.head()

,id,conversations,text,values,preprocessed_text
0,Og9h3C1,"[{'from': 'human', 'value': ""root@openvpn:/hom...",root@openvpn:/home/openvpn# ./openvpn-install....,[root@openvpn:/home/openvpn# ./openvpn-install...,rootopenvpnhomeopenvpn openvpninstallsh welcom...
1,A5AbcES,"[{'from': 'human', 'value': 'In Java, I want t...","In Java, I want to replace string like ""This i...","[In Java, I want to replace string like ""This ...",in java i want to replace string like this is ...
2,IWkMGRK,"[{'from': 'human', 'value': 'I have the follow...",I have the following C++ function: \nvoid add_...,[I have the following C++ function: \nvoid add...,i have the following c function void addplayer...
3,HskFS9I,"[{'from': 'human', 'value': '1 / 1como crear u...",1 / 1como crear un rol para kuberntes y dar so...,[1 / 1como crear un rol para kuberntes y dar s...,1 1como crear un rol para kuberntes y dar solo...
4,yn2eWCt,"[{'from': 'human', 'value': 'how do I add mult...",how do I add multiple new columns in m for pow...,[how do I add multiple new columns in m for po...,how do i add multiple new columns in m for pow...


In [11]:
corpus = df["preprocessed_text"]

In [12]:
# tokenize the documents
tokenized_documents = [doc.split() for doc in corpus]


In [13]:
# create a dictionary representation of the documents
dictionary = Dictionary(tokenized_documents)

In [14]:
# convert document into bag-of-words format
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

In [16]:
print(corpus[0])

[(0, 1), (1, 15), (2, 1), (3, 1), (4, 1), (5, 11), (6, 3), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 3), (15, 11), (16, 2), (17, 6), (18, 15), (19, 10), (20, 8), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 7), (30, 10), (31, 1), (32, 1), (33, 4), (34, 1), (35, 1), (36, 1), (37, 1), (38, 2), (39, 2), (40, 1), (41, 22), (42, 1), (43, 1), (44, 12), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 6), (51, 2), (52, 1), (53, 1), (54, 1), (55, 2), (56, 4), (57, 2), (58, 14), (59, 2), (60, 2), (61, 3), (62, 22), (63, 1), (64, 2), (65, 9), (66, 2), (67, 3), (68, 3), (69, 2), (70, 4), (71, 2), (72, 1), (73, 1), (74, 1), (75, 2), (76, 2), (77, 1), (78, 2), (79, 3), (80, 1), (81, 1), (82, 3), (83, 3), (84, 2), (85, 3), (86, 1), (87, 1), (88, 2), (89, 3), (90, 1), (91, 1), (92, 8), (93, 2), (94, 1), (95, 1), (96, 2), (97, 2), (98, 3), (99, 22), (100, 1), (101, 3), (102, 2), (103, 4), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 2)

In [57]:
lda_model = LdaModel(corpus, num_topics=2, id2word=dictionary, random_state=42)


In [58]:
topics = lda_model.print_topics()
for topic in topics:
    print(topic)

(0, '0.020*"de" + 0.014*"the" + 0.013*"la" + 0.009*"a" + 0.008*"en" + 0.007*"to" + 0.007*"const" + 0.007*"que" + 0.006*"y" + 0.006*"gt"')
(1, '0.070*"the" + 0.031*"to" + 0.029*"and" + 0.024*"a" + 0.020*"of" + 0.015*"in" + 0.013*"is" + 0.013*"for" + 0.011*"you" + 0.010*"that"')
